Oseen Equation
===

Import Netgen/NGSolve Python modules:

In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from math import log
import numpy as np
import scipy.sparse as sp
SetHeapSize(int(1e9))   

In [2]:
1/16

0.0625

In [3]:
N = 1
T = 0.2
tau = T/N 
gamma0 = 1
gamma1 = 1
gamma_dual = 1e-5
gamma_M = 1e5
#maxh = 0.0375 + (0.075-0.0375)/2
#print(maxh)
maxh = 0.8
solver = "pardiso"
#solver = "umfpack"

In [4]:
#maxh

The unit_square is a predefined domain, use Netgen to generate a mesh:

In [5]:
def freedofs_converter(fd):
    frees = [] 
    for i in range(len(fd)):
        if fd[i]:
            frees.append(i)
    return frees

def cond_est(a,frees):
    rows,cols,vals = a.mat.COO()
    A = sp.csr_matrix((vals,(rows,cols)))
    A_red = A.todense()[frees,:][:,frees] 
    return np.linalg.cond(A_red)

In [6]:
def GetMeshDataAllAround(maxh):
    geo = SplineGeometry()
    # data domain
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (1,0)
    p4 = geo.AppendPoint (0.75,0.75)
    p5 = geo.AppendPoint (0.75,0.25)
    p6 = geo.AppendPoint (0.25,0.25)
    p7 = geo.AppendPoint (0.25,0.75)
    p11 = geo.AppendPoint(1.0,1.0)
    p12 = geo.AppendPoint(0.0,1.0)
    # omega
    geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p2, p11], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p11, p12], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p12, p1], leftdomain=1, rightdomain=0,bc="bc_Omega")
    # only_B
    geo.Append (["line", p6, p5], leftdomain=2, rightdomain=1)
    geo.Append (["line", p5, p4], leftdomain=2, rightdomain=1)
    geo.Append (["line", p4, p7], leftdomain=2, rightdomain=1)
    geo.Append (["line", p7, p6], leftdomain=2, rightdomain=1)
    geo.SetMaterial(1, "omega")
    geo.SetMaterial(2, "only_B")
    return geo.GenerateMesh(maxh=maxh)

In [7]:
def GetMeshDataLeft(maxh):
    geo = SplineGeometry()
    p1 = geo.AppendPoint (0,0)
    p2 = geo.AppendPoint (0.25,0)
    p3 = geo.AppendPoint (0.25,1)
    p4 = geo.AppendPoint (0,1)
    p5 = geo.AppendPoint (1,0)
    p6 = geo.AppendPoint (1,1)
    # omega 
    geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p2, p3], leftdomain=1, rightdomain=2)
    geo.Append (["line", p3, p4], leftdomain=1, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p4, p1], leftdomain=1, rightdomain=0,bc="bc_Omega")
    # only_B 
    geo.Append (["line", p2, p5], leftdomain=2, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p5, p6], leftdomain=2, rightdomain=0,bc="bc_Omega")
    geo.Append (["line", p6, p3], leftdomain=2, rightdomain=0,bc="bc_Omega")
    geo.SetMaterial(1, "omega")
    geo.SetMaterial(2, "only_B")
    return geo.GenerateMesh(maxh=maxh)

In [8]:
#mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh = Mesh(GetMeshDataAllAround(maxh) )
# mesh = Mesh(GetMeshDataLeft(maxh))
h = specialcf.mesh_size
n = specialcf.normal(2)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [9]:
t = Parameter(0.0)

# u_sol = CoefficientFunction( (  2*x**2*y*(2*y-1)*(x-1)**2*(y-1)*exp(-t), 
#                             -2*x*y**2*(2*x-1)*(x-1)*(y-1)**2*exp(-t) ) )
# p_sol = (2*cos(x)*sin(y) - 2*sin(1)*(1 - cos(1)))*exp(-t) 
u_sol = CoefficientFunction( (  sin(3*pi*x)**2*sin(6*pi*y)*t*exp(-t**2), 
                            -sin(3*pi*y)**2*sin(6*pi*x)*t*exp(-t**2) ) )
p_sol = sin(2*pi*x)*sin(2*pi*y)*t*exp(-t) 
#beta = CoefficientFunction( ( 1.0 , 1.0 ) )
beta_ref = u_sol 
nu = 1 
u_exact = u_sol
p_exact = p_sol
rhs = CoefficientFunction(( u_exact[0].Diff(t) + (u_exact[0]*u_exact[0].Diff(x) + u_exact[1]*u_exact[0].Diff(y)) 
                           - nu* ( u_exact[0].Diff(x).Diff(x) + u_exact[0].Diff(y).Diff(y) ) +  p_exact.Diff(x) ,
                           u_exact[1].Diff(t) + (u_exact[0]*u_exact[1].Diff(x) + u_exact[1]*u_exact[1].Diff(y)) 
                           - nu* ( u_exact[1].Diff(x).Diff(x) + u_exact[1].Diff(y).Diff(y) ) +  p_exact.Diff(y)
                          ) )
# rhs = CoefficientFunction( ( 2*(-x**2*y*(x - 1)**2*(y - 1)*(2*y - 1) + 6*x**2*(1 - 2*y)*(x - 1)**2 
#               - 2*y*(y - 1)*(2*y - 1)*(x**2 + 4*x*(x - 1) + (x - 1)**2) - sin(x)*sin(y)
#               + beta_ref[0]*2*(2*y**3 - 3*y**2 + y)*(4*x**3-6*x**2+2*x)
#               + beta_ref[1]*2*(6*y**2-6*y+1)*(x**4-2*x**3+x**2) )*exp(-t)  , 
#                            2*(x*y**2*(x - 1)*(2*x - 1)*(y - 1)**2 + 2*x*(x - 1)*(2*x - 1)*(y**2 + 4*y*(y - 1) 
#                     + (y - 1)**2) + 6*y**2*(2*x - 1)*(y - 1)**2 + cos(x)*cos(y)
#                     - beta_ref[0]*2*(6*x**2-6*x+1)*(y**4-2*y**3+y**2)
#                     - beta_ref[1]*2*(2*x**3-3*x**2+x)*(4*y**3-6*y**2+2*y))*exp(-t)  )  )

# u_sol = CoefficientFunction( (   2*cos(t)*sin(pi*x)*sin(pi*x)*y*(1-y)*(1-2*y), 
#                             (-pi)*(cos(t))*sin(2*pi*x)*(y**4-2*y**3+y**2) ) )
# p_sol = sin(pi*x)*cos(pi*y)*cos(t) 
# beta = CoefficientFunction( ( 1.0 , 1.0 ) )
# rhs = CoefficientFunction( ( -2*sin(t)*sin(pi*x)*sin(pi*x)*y*(1-y)*(1-2*y)
#        -4*pi**2*cos(2*pi*x)*cos(t)*(2*y**3-3*y**2+y)-2*sin(pi*x)**2*cos(t)*(12*y-6)+pi*cos(pi*x)*cos(pi*y)*cos(t)  , 
#                            pi*sin(2*pi*x)*(y**4-2*y**3+y**2)*sin(t)
#       -(4*pi**3*sin(2*pi*x)*(y**4-2*y**3+y**2)*cos(t)-pi*sin(2*pi*x)*(12*y**2-12*y+2)*cos(t))
#       -pi*sin(pi*x)*sin(pi*y)*cos(t) )  )



Define a finite element space on that mesh. 

In [10]:
#fes = H1(mesh, order=3, dirichlet="left|right|bottom|top")
fes_NC = FESpace("nonconforming",mesh, dirichlet="bc_Omega", dgjumps = True) 
fes_lam = NumberSpace(mesh)
fes_L2 = L2(mesh, order=0)
fes_primal_vel = FESpace([fes_NC*fes_NC for i in range(N+1) ])
fes_primal_pressure = FESpace([ fes_L2 for i in range(N+1) ])
fes_dual_vel = FESpace([fes_NC*fes_NC for i in range(N+1) ])
fes_dual_pressure = FESpace([ fes_L2 for i in range(N+1) ])
fes_primal_lam = FESpace([fes_lam for i in range(N)])
fes_dual_lam = FESpace([fes_lam for i in range(N)])
X = FESpace( [fes_primal_vel, fes_primal_pressure,fes_primal_lam, fes_dual_vel, fes_dual_pressure,fes_dual_lam])
print ("X-ndof = {0}".format(X.ndof ))

X-ndof = 180


In [11]:
#gfu = GridFunction(X)
#Draw( gfu.components[0].components[N].components[1], mesh)
#print(gfu.components[0].dim)
#help(gfu)
#help(gfu.components[1])
#print(gfu.components[0][0].FV().NumPy())
#print(gfu.components[0][0].dim)
#help( gfu.components[0][0])
#print(len(gfu.components[0][0] ))
#Draw(gfu.components[1],mesh )

In [12]:
#help(gfu.components[0].components[0].components[0])

In [13]:
len(X.TrialFunction()) 

6

In [14]:
u, pp, llam, zz, yyy, xxi =  X.TrialFunction()
v, qq, mmu, ww, xxx, eeta =  X.TestFunction()
#print(len(u[0]) )
#print

p = [pp[i] for i in range(len(pp)) ]
z = [zz[i] for i in range(len(zz)) ]
yy = [yyy[i] for i in range(len(yyy)) ]
lam = [None] + [llam[i] for i in range(len(llam)) ]
xi = [None] + [xxi[i] for i in range(len(xxi)) ]

q = [qq[i] for i in range(len(qq)) ]
w = [ww[i] for i in range(len(ww)) ]
xx = [xxx[i] for i in range(len(xxx)) ]
mu = [None] + [mmu[i] for i in range(len(mmu)) ]
eta = [None] + [eeta[i] for i in range(len(eeta)) ]

#help(xx)

In [15]:
def IP(u,v,nabla=False):
    if nabla:
        return sum( [ grad(u[i])*grad(v[i]) for i in range(len(u))] )
    else:
        return sum( [u[i]*v[i] for i in range(len(u))] )

def IP_conv(u,v,beta):
    return beta[0]*grad(u[0])[0]*v[0]+beta[1]*grad(u[0])[1]*v[0]+beta[0]*grad(u[1])[0]*v[1]+beta[1]*grad(u[1])[1]*v[1]
    
def IP_ut_v(u_cur,u_prev,v):
    return sum( [ (u_cur[i] - u_prev[i] ) * v[i] for i in range(len(u_cur))] )
    #help(IP(u[0],v[0]))
def IP_mixed_stab(u_cur,u_prev,v_cur,v_prev):
    return sum( [ ( grad(u_cur[i]) - grad(u_prev[i]) ) * ( grad(v_cur[i]) - grad(v_prev[i]) )  for i in range(len(u_cur))] )

def IP_CIP(u,v):
    return sum( [ (u[i] - u[i].Other()) * (v[i] - v[i].Other()) for i in range(len(u))  ] )

def IP_jump_avg(u,v):
    return sum( [ (u[i] - u[i].Other()) * 0.5 * (v[i] + v[i].Other()) for i in range(len(u))  ] )

def IP_divu_q(u,q):
    u1_dx = grad(u[0])[0]
    u2_dy = grad(u[1])[1]
    div_u = u1_dx + u2_dy
    return div_u * q  

In [16]:
gfu = GridFunction(X)
beta_prev = GridFunction(X)
#beta_prev.vec.FV().NumPy()[:] = 1e1*np.random.rand(len( beta_prev.vec.FV().NumPy() ))[:]
beta_prev.vec.FV().NumPy()[:] = 0.0

In [17]:
def SolveOseen(beta_prev):
    
    a = BilinearForm(X,symmetric=False)

    # add mean value pressure constraint 
    for i in range(1,N+1):
        a += (mu[i] * p[i] + lam[i] * q[i]) * dx  
        a += (eta[i] * yy[i] + xi[i] * xx[i]) * dx 
        
    # divergence zero constraint for initial data
    a += (-1)*IP_divu_q(w[0],p[0]) * dx
    a += IP_divu_q(u[0],xx[0]) * dx 
    a += (-1)*IP_divu_q(z[0],q[0]) * dx 
    a += IP_divu_q(v[0],yy[0]) * dx

    # A1 
    for i in range(1,N+1):
        beta = CoefficientFunction((beta_prev.components[0].components[i-1].components[0], beta_prev.components[0].components[i-1].components[1] ) ) 
        
        a += IP_ut_v(u[i],u[i-1],w[i]) * dx
        a += tau * IP(u[i],w[i],nabla=True) * dx 
        a += tau * IP_conv(u[i], w[i],beta) * dx 
        a += tau *(-1)*IP_divu_q(w[i],p[i]) * dx
        a += tau * IP_divu_q(u[i],xx[i]) * dx 
        a +=  (-1)*tau * InnerProduct(beta , n ) * IP_jump_avg(u[i],w[i]) * dx(skeleton=True)
        #a +=  tau * (0.5*IfPos(InnerProduct(beta,n),InnerProduct(beta,n),-InnerProduct(beta,n))) * IP_CIP(u[i],w[i]) * dx(skeleton=True)

    # A2 
    a += gamma0 * h**2 * IP(u[0],v[0],nabla=True) * dx
    for i in range(1,N+1):
        beta = CoefficientFunction((beta_prev.components[0].components[i-1].components[0], beta_prev.components[0].components[i-1].components[1] ) ) 
    #     a += gamma_M * tau * IP(u[i],v[i]) * dx(definedon=mesh.Materials("omega"))
        a += gamma_M * tau * IP(u[i],v[i]) * dx(definedon=mesh.Materials("only_B"))
        a += gamma1 * tau * IP_mixed_stab(u[i],u[i-1],v[i],v[i-1]) * dx 
        a +=  tau * (1/h)  * IP_CIP(u[i],v[i]) * dx(skeleton=True)
        a +=  (-1)*tau * InnerProduct(beta , n ) * IP_jump_avg(v[i],z[i]) * dx(skeleton=True)
        #a +=  tau * (0.5*IfPos(InnerProduct(beta,n),InnerProduct(beta,n),-InnerProduct(beta,n))) * IP_CIP(v[i],z[i]) * dx(skeleton=True)
        a += IP_ut_v(v[i],v[i-1],z[i]) * dx
        a += tau * IP(v[i],z[i],nabla=True) * dx 
        a += tau * IP_conv(v[i], z[i],beta) * dx 

        a += (-1)*IP_divu_q(z[i],q[i]) * dx 
        a += IP_divu_q(v[i],yy[i]) * dx
        
    f = LinearForm(X)
    for i in range(1,N+1):
        t.Set(tau*i)
        f += tau * IP(w[i],rhs,nabla=False)  * dx
        f +=  gamma_M * tau * IP(v[i],u_sol) * dx(definedon=mesh.Materials("only_B"))

    with TaskManager():
        a.Assemble()
        f.Assemble()
    
    gfu.vec.data = a.mat.Inverse(X.FreeDofs(),inverse=solver) * f.vec
    
    uhx = gfu.components[0].components[N].components[0]
    uhy = gfu.components[0].components[N].components[1]
    uh = [uhx,uhy]
    p_primal = gfu.components[1].components[N]
    p_dual = gfu.components[4].components[N]
    
    t.Set(tau*N)
    Draw (u_sol[0] , mesh);
    
    Draw (uhx, mesh);
    
    # compute error 
    t.Set(tau*N)
    error=sqrt(Integrate( (u_sol[0]-uhx)**2 + (u_sol[1]-uhy)**2 , mesh))
    ref_error=sqrt(Integrate( (u_sol[0])**2 + (u_sol[1])**2 , mesh))
    print ("L2-error velocity final time:", error)
    error_pressure=sqrt(Integrate( (p_sol-p_primal)**2  , mesh))
    print("L2-error pressure final time:", error_pressure)
#     print ("L2-error:", error/ref_error)
    beta_prev.vec.data = gfu.vec.data 
    
    # error computation for velocity at initial time
    t.Set(0*N)
    uhx0 = gfu.components[0].components[0].components[0]
    uhy0 = gfu.components[0].components[0].components[1]
    error0 =sqrt(Integrate( (u_sol[0]-uhx0)**2 + (u_sol[1]-uhy0)**2 , mesh))
    print ("L2-error velocity initial time:", error0)
    t.Set(tau*N)
    
    return error
#     return error/ref_error
    

In [18]:
l2_errors = [ ]
# Fixed point iteration 
maxiter = 5 
for i in range(maxiter): 
    l2err = SolveOseen(beta_prev)
    l2_errors.append(l2err) 

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

L2-error velocity final time: 0.33199080557914673
L2-error pressure final time: 3.58861876444008
L2-error velocity initial time: 5.796166816923844


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

L2-error velocity final time: 0.34370660731007446
L2-error pressure final time: 3.4023229385660656
L2-error velocity initial time: 5.762115115403878


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

L2-error velocity final time: 0.3434814015100059
L2-error pressure final time: 3.402433068653031
L2-error velocity initial time: 5.767744934536913


In [19]:
l2_errors

[0.33199080557914673,
 0.3466980686020546,
 0.3429446567308939,
 0.34370660731007446,
 0.3434814015100059]

In [20]:
print([ abs(l2_errors[i] - l2_errors[-1] ) for i in range(len(l2_errors)-1 ) ])

[0.01149059593085916, 0.0032166670920487084, 0.000536744779111975, 0.00022520580006857305]


In [21]:
# Oseen pardiso failed for N=16
# l2_errors = [0.8250961364400698,0.9787851609597009,0.34084671037149683, 0.10168199089645942 ]
l2_errors = [0.3434814015100059, 0.6079334237794947,0.184076047273199 ,0.031714512206010265,0.007896476188968517]
#hm = [ 0.8,0.4,0.2,0.1,0.05]
#Ns = [1,2,4,8,16]
eoc = [ log(l2_errors[i-1]/l2_errors[i])/log(2) for i in range(1,len(l2_errors))]
print(eoc)

[-0.8236813558496889, 1.7236114298568461, 2.537086851855437, 2.00586225477592]


In [22]:
L2-error final time: 0.3434814015100059
L2-error pressure final time: 3.402433068653031
L2-error initial time: 5.767744934536913

L2-error velocity final time: 0.6079334237794947
L2-error pressure final time: 1.9289243749343423
L2-error velocity initial time: 4.1372481557270975

L2-error velocity final time: 0.184076047273199
L2-error pressure final time: 0.6869987249544074
L2-error velocity initial time: 0.6551807876484467

L2-error velocity final time: 0.031714512206010265
L2-error pressure final time: 0.38896990232674644
L2-error velocity initial time: 0.08613151908362635
    
    
L2-error velocity final time: 0.007896476188968517
L2-error pressure final time: 0.17212553594103372
L2-error velocity initial time: 0.09770247438213957

SyntaxError: invalid syntax (3742224666.py, line 1)